In [1]:
import pandas as pd
from cleaning import create_demand
import pyomo.environ as pyo
import pyomo
from pandas import DataFrame


Lightning Network nodes can have multiple type of connections to be used.
Specifically, every node can be an hidden node - thus leveraging privacy
preserving communication protocols like Tor or I2P - or can use a clearnet connection,
with a standard IPv4 or IPv6 (or both) address.

In order to assign the "connection type" feature to each node, the following mapping
has been implemented:

- 1 if the node uses a privacy preserving protocol, eg it works as an hidden service;
- 2 if the node is a clearned with an IPv4 or IPv6 address;
- 3 if the node uses both connection typologies (eg both clearnet and onion)
 
Note that the score is independent from the number of addresses of each kind.
Thus if a node where to
have both an IPv4 and IPv6, the score would be 2 despite the support to two clearnet protocols.

The goal of this second scenario is to compute the shortest path with minimum cost between two nodes,
considering that privacy preserving nodes are, on average, more prone to higher latency, thus
causing payment failures and thus increasing the payment time.

In order to reproduce this scenario, we assign a penalization to the usage of channels that have a
connection type different from 3, which is the connection type associated with a wider range
of communication protocols and thus is likely to be associated with lower latency or less connection issues.

##### Alternative formulation

Setup the problem considering a constraint such that the address code has to be at least 2, meaning that we avoid nodes who have an onion only connection because they are associated to higher latency.

# Data import

In [2]:
nodes = pd.read_pickle("../../data/original/nodes.pkl")
channels = pd.read_pickle("../../data/original/channels.pkl")

In [3]:
def drop_not_mapped(df_nodes: DataFrame, df_channels: DataFrame) -> DataFrame:
    """
    Drop channels that have at least one peer not appearing in the nodes dataframe.
    :param df_nodes: nodes dataframe
    :param df_channels: channels dataframe
    :return: filtered channels dataframe containing only the channels
    whose peers partecipants both appear in the nodes dataframe
    """
    df_channels = df_channels[
        df_channels.loc[:, "node1_pub"].isin(df_nodes.index) &
        df_channels.loc[:, "node2_pub"].isin(df_nodes.index)
        ]
    return df_channels


def drop_missing_addresses(df_nodes: DataFrame) -> DataFrame:
    """
    Drop the nodes that have an unknown address type (which is 0).
    :param df_nodes: nodes dataframe
    :return: nodes dataframe without the nodes with address ==0
    """
    return df_nodes.loc[df_nodes.loc[:, "addresses"] != 0, :]


#channels = drop_not_mapped(nodes, channels)
nodes = drop_missing_addresses(nodes)

## Modeling

In [4]:
pyomo.common.timing.report_timing()

In [5]:
# Multiplication to set the same base unit for all computations
channels["capacity"] = channels["capacity"] * 1000000
channels["base_fee"] = channels["base_fee"] * 1000000

In [6]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

           0 seconds to construct Block ConcreteModel; 1 index total
        0.02 seconds to construct Set NODES; 1 index total
        0.30 seconds to construct Set CHANNELS; 1 index total


In [7]:
nodes = create_demand(nodes, 500000,
                      source="02b4098c3116124c0c4105295d3d2e65927d46e98e248d84cb6119da57e4ae31e3",destination="0202f00d5f5c91a3c79113851b162c2b75f2cbd9fb2378fb4a4ce92d5ba1e63e08")

Transaction of 500000 sats.
Sender: VeniceForever
Receiver: ShatSat412.


In [8]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

        0.11 seconds to construct Var x; 89898 indices total
        0.12 seconds to construct Var a; 89898 indices total


In [18]:
model.nc = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(2, 3))

'pyomo.core.base.var.IndexedVar'>) on block Min cost flow problem with a new
Component (type=<class 'pyomo.core.base.var.IndexedVar'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
        0.07 seconds to construct Var nc; 89898 indices total


In [19]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

### Objective function

$$
min \sum_{i \in E} ^{} amount_i \times feerate_{i} + x_{i} \times basefee_{i} + x_{i} \times \frac{1}{min(addresses_{node1_{i}}, addresses_{node2_{i}})}
$$

Note that here we need to handle also the situation in which a node has only one channel, thus
its pubkey appears only as node1_pub (or node2_pub). Thus we need an exception handling when looking
for the minimum between the two addresses scores.
Note also that we consider the minimum between the two addresses scores because we want to penalize
the lower scores. Since we are minimizing the objective function, we assign a 1/min_score as a penalization coefficient

In [11]:
# The following code adds a column to the channels dataframe which contains the min score of addresses for each channel
channels.loc[:, "addresses"] = 0.0001
for i in channels.index:
    try:
        channels.at[i, "addresses"] = min([nodes.loc[i[0], "addresses"], nodes.loc[i[1], "addresses"]])
    except KeyError:
        try:
            channels.at[i, "addresses"] = nodes.loc[i[0], "addresses"]
        except KeyError:
            try:
                channels.at[i, "addresses"] = nodes.loc[i[1], "addresses"]
            except:
                pass


In [12]:
def objective_function(model: pyo.ConcreteModel):
    return sum(
        model.a[i] * channels.loc[i, "rate_fee"] +
        model.x[i] * channels.loc[i, "base_fee"] +
        model.x[i] * 1/channels.loc[i, "addresses"]
        for i in model.CHANNELS)

#def objective_function(model: pyo.ConcreteModel):
#    """
#    Note that here we need to handle also the situation in which a node has only one channel, thus
#    its pubkey appears only as node1_pub (or node2_pub). Thus we need an exception handling when looking
#    for the minimum between the two addresses scores.
#    Note also that we consider the minimum between the two addresses scores because we want to penalize
#    the lower scores. Since we are minimizing the objective function, we assign a 1/min_score as a penalization coefficient
#    """
#    return sum(
#        model.a[i] * channels.loc[i, "rate_fee"] +
#        model.x[i] * channels.loc[i, "base_fee"] +
#        model.x[i] * (
#            1 / min(
#                (nodes.loc[i[0], "addresses"], nodes.loc[i[1], "addresses"])#,
#                #default=float('inf')
#            )
#            if (
#                   i[0] in nodes.index and i[1] in nodes.index and
#                   pd.notnull(nodes.loc[i[0], "addresses"]) and
#                   pd.notnull(nodes.loc[i[1], "addresses"])
#            )
#            else
#            (
#                nodes.loc[i[0], "addresses"] if pd.notnull(nodes.loc[i[0], "addresses"]) else (
#                    nodes.loc[i[1], "addresses"] if pd.notnull
#                else 0.0001
#                )
#            )
#        )
#        for i in model.CHANNELS
#    )


model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

           0 seconds to construct Objective totalCost; 1 index total


### Constraints

#### Symmetric channel flow rule

This constraint enforces the symmetry between the number of channel incoming and outgoing from an intermediary node (excluding source and destination nodes)

$$
\sum_{(i,n) \in E} x_{i,n} = \sum_{(n,j) \in E} x_{n,j} \text{ } \forall n \in V \bigwedge i \ne \{n, s, d\}
$$

where $s$ is the source node, with a negative demand and $d$ is the destination node, with positive demand.

In [13]:
s = nodes[nodes["demand"] < 0].index.values
d = nodes[nodes["demand"] > 0].index.values
intermediaries = [(i, j) for i, j in channels.index if i != s and j != d]

In [14]:
def number_channels_rule(model: pyo.ConcreteModel, n):
    incoming = [model.x[(i, j)] for i, j in intermediaries if j == n]
    outgoing = [model.x[(i, j)] for i, j in intermediaries if i == n]
    return sum(incoming) == sum(outgoing)

model.NumberChannelsConstraint = pyo.Constraint(model.NODES, rule=number_channels_rule, name="Number channels constraint")

       49.81 seconds to construct Constraint NumberChannelsConstraint; 11312 indices total


#### Connection Constraint

$$
Connection_{i,j} \ge connectionScore_{i,j} \times x_{i,j} \forall(i,j) \in E
$$

In [22]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.nc[(a, b)] >= channels.loc[(a, b), "addresses"] * model.x[(a, b)]

model.ConnectionConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name='Connection constraint')

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block Min
cost flow problem with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
        5.81 seconds to construct Constraint ConnectionConstraint; 89898 indices total


#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$

In [21]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <= channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

        4.80 seconds to construct Constraint CapacityConstraint; 89898 indices total


#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{s,i} - \sum_{(i,t) \in E} amount_{i,d} = b_i \text{ } \forall i \in V$$

where $s$ is the source node, $d$ is the destination node, $i$ is every intermediary node


In [ ]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True) 

#### Avoid Onion Nodes constraint

$$

$$

## Solving the model

In [ ]:
from pyomo.common.tempfiles import TempfileManager
#TempfileManager.tempdir = "/home/filippo/repositories/LN-pathfinding"

In [ ]:
opt = pyo.SolverFactory('cbc')
#opt = pyo.SolverFactory('/opt/ibm/ILOG/CPLEX_Studio_Community2212/cplex/bin/x86-64_linux/cplex')
results = opt.solve(model, tee=True, keepfiles=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())


In [ ]:
from decimal import Decimal
pd.set_option('display.max_colwidth', 10000000000000000000000000000000000000000000000000000)
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    if value > 0.1:
        DF_channels.loc[c, "source"] = index[0]
        DF_channels.loc[c, "destination"] = index[1]
        try:
            DF_channels.loc[c, "source-alias"] = nodes.loc[index[0], "alias"]
        except KeyError:
            print("Source alias not found in nodes database: placeholder inserted")
            DF_channels.loc[c, "source-alias"] = "source-alias"
            pass
        try:
            DF_channels.loc[c, "destination-alias"] = nodes.loc[index[1], "alias"]
        except KeyError:
            print("Destination alias not found in nodes database: placeholder inserted")
            DF_channels.loc[c, "destination-alias"] = "destination-alias"
            pass
        DF_channels.loc[c, "capacity"] = Decimal(channels.loc[index, "capacity"]/1000000)
        DF_channels.loc[c, "amount"] = Decimal(value/1000000)
        DF_channels.loc[c, "base_fee"] = Decimal(channels.loc[(index[0], index[1]), "base_fee"]/1000000)
        DF_channels.loc[c, "rate_fee"] = channels.loc[(index[0], index[1]), "rate_fee"]
        #DF_channels.loc[c, "used_fl"] = channels.loc[]
        c += 1

DF_channels_pos = DF_channels[DF_channels["amount"]!=0]

In [ ]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

In [ ]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    if value != 0:
        DF_fixed.loc[c, "source"] = index[0]
        DF_fixed.loc[c, "destination"] = index[1]
        DF_fixed.loc[c, "used"] = Decimal(value)
        c += 1

DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]
DF_fixed_pos

### All data about channels on path

In [ ]:
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
final = intersection[intersection["amount"]>0]
intersection[intersection["amount"]>0]

In [ ]:
from collections import Counter
#results = [tuple(i) for i in DF_channels_pos[["source-alias", "destination-alias"]].to_numpy()]
results = [tuple(i) for i in final[["source-alias", "destination-alias"]].to_numpy()]
unwrap = [e for i in results for e in i]
extremes = [j for j in unwrap if Counter(unwrap)[j] == 1]
def return_source(l: list):
    if l[0] in [i for i, j in results]:
        return l[0]
    else: 
        return l[1]
sequence = []        
source = return_source(extremes)
sequence.append(source)
while len(sequence) <= len(results):
    for i,j in results:
        if i == sequence[-1]:
            sequence.append(j)
for i in sequence:
    print("⬇️", i)